# 2048
A fun game to play in a not so fun way - through the command line.

In [1]:
import random
import time

from IPython.display import clear_output
import numpy as np

In [2]:
# GLOBAL VARS
UPPER_ROW = LEFT_COL = 0

LOWER_ROW = 3
RIGHT_COL = 3
NUM_OF_BOARD_SPACES = (LOWER_ROW + 1) * (RIGHT_COL + 1)

PROBABILITY_OF_SPAWNING_4 = 0.1

SCORE = MOVE_COUNTER = 0

MOVE_MADE = None

BOARD = np.zeros((LOWER_ROW + 1, RIGHT_COL + 1)).astype(int)

## What's Left to Do:
* More verbose debug mode?
* Tracking of all moves made by user, score, move count at time, etc.
* Add documentation to functions.
* Get this in a `.py` file.

## Game Setup: 

In [3]:
def add_random_space():
    global BOARD

    list_of_zeros = list(zip(np.where(BOARD == 0)[0], np.where(BOARD == 0)[1]))
    if list_of_zeros:
        if random.random() < PROBABILITY_OF_SPAWNING_4:
            BOARD[random.choice(list_of_zeros)] = 4
        else:
            BOARD[random.choice(list_of_zeros)] = 2
    else:
        # TODO: raise error of some sort.
        print('You lose!')

In [4]:
def recursively_move(row, col, direction, combine=False):
    global BOARD, SCORE, MOVE_MADE
    
    # ensure we have proper checks and adjusted rows/cols for the specified direction
    adjusted_row_dict = {'s': row+1, 'w': row-1, 'd': row, 'a': row}
    adjusted_col_dict = {'s': col, 'w': col, 'd': col+1, 'a': col-1}
    adjusted_row = adjusted_row_dict[direction]
    adjusted_col = adjusted_col_dict[direction]

    if (direction == 's'
        and (row < UPPER_ROW or col < LEFT_COL
             or row > LOWER_ROW - 1 or col > RIGHT_COL)
        ):
        return
    elif (direction == 'w'
          and (row - 1 < UPPER_ROW or col < LEFT_COL
               or row > LOWER_ROW or col > RIGHT_COL)
          ):
        return
    elif (direction == 'd'
          and (row < UPPER_ROW or col < LEFT_COL
               or row > LOWER_ROW or col > RIGHT_COL - 1)
          ):
        return
    elif (direction == 'a'
          and (row < UPPER_ROW or col - 1 < LEFT_COL
               or row > LOWER_ROW or col > RIGHT_COL)
          ):
        return
    
    # now general logic
    elif BOARD[row, col] == 0:
        return
    elif combine is False and BOARD[adjusted_row, adjusted_col] == 0:
        BOARD[adjusted_row, adjusted_col] = BOARD[row, col]
    elif combine is True and BOARD[row, col] == BOARD[adjusted_row, adjusted_col]:
        BOARD[adjusted_row, adjusted_col] *= 2
        SCORE += BOARD[adjusted_row, adjusted_col]
    else:
        return

    BOARD[row, col] = 0
    MOVE_MADE = True
    if combine is False:
        recursively_move(adjusted_row, adjusted_col, direction, combine=False)

In [5]:
def shift(direction):
    global MOVE_COUNTER, MOVE_MADE
    
    if MOVE_MADE is None or MOVE_MADE is True:
        MOVE_MADE = False
    
    if direction == 's':
        # down shift everything twice, combine only once
        for iteration in range(2):
            for row_start in range(LOWER_ROW - 1, UPPER_ROW - 1, -1):
                for row in range(row_start, UPPER_ROW - 1, -1):
                    for col in range(LEFT_COL, RIGHT_COL + 1):
                        recursively_move(row, col, direction, combine=False)
            # only combine once
            if iteration == 0:
                for row in range(LOWER_ROW - 1, UPPER_ROW - 1, -1):
                    for col in range(LEFT_COL, RIGHT_COL + 1):
                        recursively_move(row, col, direction, combine=True)
    elif direction == 'd':
        # right shift everything twice, combine only once
        for iteration in range(2):
            for col_start in range(RIGHT_COL - 1, LEFT_COL - 1, -1):
                for col in range(col_start, LEFT_COL - 1, -1):
                    for row in range(UPPER_ROW, LOWER_ROW + 1):
                        recursively_move(row, col, direction, combine=False)
            # only combine once
            if iteration == 0:
                for col in range(RIGHT_COL - 1, LEFT_COL - 1, -1):
                    for row in range(UPPER_ROW, LOWER_ROW + 1):
                        recursively_move(row, col, direction, combine=True)
    elif direction == 'w':
        # up shift everything twice, combine only once
        for iteration in range(2):
            for row_start in range(UPPER_ROW + 1, LOWER_ROW + 1):
                for row in range(row_start, LOWER_ROW + 1):
                    for col in range(LEFT_COL, RIGHT_COL + 1):
                        recursively_move(row, col, direction, combine=False)
            # only combine once
            if iteration == 0:
                for row in range(UPPER_ROW + 1, LOWER_ROW + 1):
                    for col in range(LEFT_COL, RIGHT_COL + 1):
                        recursively_move(row, col, direction, combine=True)
    elif direction == 'a':
        # left shift everything twice, combine only once
        for iteration in range(2):
            for col_start in range(LEFT_COL + 1, RIGHT_COL + 1):
                for col in range(col_start, RIGHT_COL + 1):
                    for row in range(UPPER_ROW, LOWER_ROW + 1):
                        recursively_move(row, col, direction, combine=False)
            # only combine once
            if iteration == 0:
                for col in range(LEFT_COL + 1, RIGHT_COL + 1):
                    for row in range(UPPER_ROW, LOWER_ROW + 1):
                        recursively_move(row, col, direction, combine=True)
    else:
        # TODO: raise error of some sort.
        print('ERROR')

    MOVE_COUNTER += 1

In [6]:
def check_if_there_is_a_move_left():    
    # are there any 0s left?
    if len(BOARD.nonzero()[0]) < NUM_OF_BOARD_SPACES:
        return True
    # if not, can we move any blocks over potentially in the coming moves?
    for row in range(UPPER_ROW, LOWER_ROW + 1):
        for col in range(LEFT_COL, RIGHT_COL + 1):
            if can_we_move_this_block(row, col):
                return True
    return False


def can_we_move_this_block(row, col):
    if col > LEFT_COL:
        if BOARD[row, col] == BOARD[row, col - 1]:
            return True
    if col < RIGHT_COL:
        if BOARD[row, col] == BOARD[row, col + 1]:
            return True
    if row > UPPER_ROW:
        if BOARD[row, col] == BOARD[row - 1, col]:
            return True
    if row < LOWER_ROW:
        if BOARD[row, col] == BOARD[row + 1, col]:
            return True
    else:
        return False

In [7]:
def reset():
    global SCORE, MOVE_COUNTER, MOVE_MADE, BOARD

    SCORE = 0
    MOVE_COUNTER = 0
    MOVE_MADE = None
    BOARD = np.zeros((LOWER_ROW + 1, RIGHT_COL + 1)).astype(int)
    
    # add in some random spaces.
    for _ in range(2):
        add_random_space()

In [8]:
def transform_direction_input():
    raw_direction = input('Enter a direction [w/a/s/d or down/up/right/left]: ')
    direction = raw_direction.strip().lower()

    if direction == 'quit' or direction == 'q' or direction == 'exit':
        raise KeyboardInterrupt('Goodbye!')
    elif direction == 'right':
        direction = 'd'
    elif direction == 'left':
        direction = 'a'
    elif direction == 'up':
        direction = 'w'
    elif direction == 'down':
        direction = 's'
    if direction in ['w', 'a', 's', 'd']:
        return direction

    print(f'"{raw_direction}" is not valid. Try again or type "quit".')
    return transform_direction_input()

In [9]:
def pretty_print(mat, fmt="n"):
    """
    Modified from https://gist.github.com/braingineer/d801735dac07ff3ac4d746e1f218ab75.
    `n` for number.
    """
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        print('     ', end='')
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end='  ')
        print()

## Tests 

In [10]:
# Test that we can correctly detect when legal moves remain.
BOARD = np.reshape(np.arange(0, 16), (4, 4))
assert check_if_there_is_a_move_left() == True
BOARD = np.reshape(np.arange(1, 17), (4, 4))
assert check_if_there_is_a_move_left() == False
BOARD = np.reshape(np.ones(16), (4, 4))
assert check_if_there_is_a_move_left() == True
reset()
assert check_if_there_is_a_move_left() == True

In [11]:
# Test shift down
BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('s')
assert np.array_equal(BOARD, np.array([[0, 0, 0, 0],
                                       [0, 0, 0, 0],
                                       [4, 0, 0, 0],
                                       [4, 2, 2, 2]]))

In [12]:
# Test shift down special consideration #1
BOARD = np.array([[2, 2, 2, 2],
                  [0, 0, 0, 0],
                  [2, 0, 0, 0],
                  [4, 0, 0, 0]])
shift('s')
assert np.array_equal(BOARD, np.array([[0, 0, 0, 0],
                                       [0, 0, 0, 0],
                                       [4, 0, 0, 0],
                                       [4, 2, 2, 2]]))

In [13]:
# Test shift down special consideration #2
BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [4, 0, 0, 0],
                  [8, 0, 0, 0]])
shift('s')
assert np.array_equal(BOARD, np.array([[0, 0, 0, 0],
                                       [4, 0, 0, 0],
                                       [4, 0, 0, 0],
                                       [8, 2, 2, 2]]))

In [14]:
# Test shift up
BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('w')
assert np.array_equal(BOARD, np.array([[4, 2, 2, 2],
                                       [4, 0, 0, 0],
                                       [0, 0, 0, 0],
                                       [0, 0, 0, 0]]))

In [15]:
# Test shift up special consideration #1
BOARD = np.array([[2, 2, 2, 2],
                  [0, 0, 0, 0],
                  [2, 0, 0, 0],
                  [4, 0, 0, 0]])
shift('w')
assert np.array_equal(BOARD, np.array([[4, 2, 2, 2],
                                       [4, 0, 0, 0],
                                       [0, 0, 0, 0],
                                       [0, 0, 0, 0]]))

In [16]:
# Test shift up special consideration #2
BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [4, 0, 0, 0],
                  [8, 0, 0, 0]])
shift('w')
assert np.array_equal(BOARD, np.array([[4, 2, 2, 2],
                                       [4, 0, 0, 0],
                                       [8, 0, 0, 0],
                                       [0, 0, 0, 0]]))

In [17]:
# Test shift right
BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('d')
assert np.array_equal(BOARD, np.array([[0, 0, 4, 4],
                                       [0, 0, 0, 2],
                                       [0, 0, 0, 2],
                                       [0, 0, 0, 2]]))

In [18]:
# Test shift right special consideration #1
BOARD = np.array([[2, 0, 2, 4],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('d')
assert np.array_equal(BOARD, np.array([[0, 0, 4, 4],
                                       [0, 0, 0, 2],
                                       [0, 0, 0, 2],
                                       [0, 0, 0, 2]]))

In [19]:
# Test shift right special consideration #2
BOARD = np.array([[2, 2, 4, 8],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('d')
assert np.array_equal(BOARD, np.array([[0, 4, 4, 8],
                                       [0, 0, 0, 2],
                                       [0, 0, 0, 2],
                                       [0, 0, 0, 2]]))

In [20]:
# Test shift left
BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('a')
assert np.array_equal(BOARD, np.array([[4, 4, 0, 0],
                                       [2, 0, 0, 0],
                                       [2, 0, 0, 0],
                                       [2, 0, 0, 0]]))

In [21]:
# Test shift right special consideration #1
BOARD = np.array([[2, 0, 2, 4],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('a')
assert np.array_equal(BOARD, np.array([[4, 4, 0, 0],
                                       [2, 0, 0, 0],
                                       [2, 0, 0, 0],
                                       [2, 0, 0, 0]]))

In [22]:
# Test shift right special consideration #2
BOARD = np.array([[2, 2, 4, 8],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('a')
assert np.array_equal(BOARD, np.array([[4, 4, 8, 0],
                                       [2, 0, 0, 0],
                                       [2, 0, 0, 0],
                                       [2, 0, 0, 0]]))

In [23]:
# Test `reset()`, `MOVE_COUNTER`, and `SCORE` values
reset()
assert MOVE_COUNTER == 0
assert SCORE == 0
assert MOVE_MADE == None
assert len(BOARD.nonzero()[0]) == 2

BOARD = np.array([[2, 2, 2, 2],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0],
                  [2, 0, 0, 0]])
shift('a')
shift('d')
shift('w')
shift('s')
assert MOVE_COUNTER == 4
assert SCORE == 20
assert MOVE_MADE == True

reset()
assert MOVE_COUNTER == 0
assert SCORE == 0
assert MOVE_MADE == None
assert len(BOARD.nonzero()[0]) == 2

In [24]:
# Test `MOVE_MADE` doesn't update on bad moves
reset()
BOARD = np.array([[2, 4, 8, 16],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0]])
shift('d')
shift('a')
assert MOVE_MADE is False

## The Actual Game 

In [25]:
def play(debug=False):
    global BOARD, SCORE, MOVE_COUNTER, MOVE_MADE
    
    try:
        # Start by setting up the board.
        reset()

        while True:
            if debug is False:
                clear_output()

            if MOVE_MADE is False:
                print("Move not made! Try again.")
                MOVE_COUNTER -= 1

            print(f'Score: {int(SCORE)}, Move Counter: {MOVE_COUNTER}')
            print('-------------------------')
            print()

            if debug is False:
                pretty_print(BOARD)
            else:
                print(BOARD)
            print()

            # Check to make sure there is a move left we can make.
            if check_if_there_is_a_move_left() is False:
                print(f'You Lose! Final Score: {SCORE}')
                return

            # Now ask the user for input, then send that to shift.
            direction = transform_direction_input()
            shift(direction)
            if MOVE_MADE:
                # Add another random piece to the board.
                add_random_space()

    except Exception as e:
        print(f'Uh-oh! Something went horribly wrong! This is not good.',
               'Here is some specifics on what went so poorly: ')
        print(e.message, e.args)

In [ ]:
play()

Score: 424, Move Counter: 53
-------------------------

      2   0  0  0  
      8   0  0  0  
     16   4  0  2  
     64  16  4  4  



-----